In [1]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import cv2

In [2]:
images = {
    size * 10: np.load(f"data/photo_calibrate_{size}.npy") for size in [50, 100, 150, 200, 250, 300, 350, 400, 500, 600]
}
len(images)

10

In [3]:
canny_images = [
    cv2.Canny(cv2.blur(img, (3, 3)), 10, 70, apertureSize=3) for img in images.values()
]
# px.imshow(np.array(canny_images), animation_frame=0, width=5_00).show()

In [4]:
arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
arucoParams = cv2.aruco.DetectorParameters()
(corners, ids, rejected) = cv2.aruco.detectMarkers(
    images[5_000], arucoDict, parameters=arucoParams
)
# px.imshow(cv2.aruco.drawDetectedMarkers(images[0], corners, ids))

In [5]:
# 14.3, 14.4 cm shape
marker_size_mm = 144


def guess_focal(img: np.array, dist_mm: int, size_mm: int = marker_size_mm):
    cs = cv2.aruco.detectMarkers(img, arucoDict, parameters=arucoParams)[0][0][0]
    pixel_size = np.mean(
        [
            np.sqrt(np.square(cs[0] - cs[3]).sum()),
            np.sqrt(np.square(cs[1] - cs[2]).sum()),
        ]
    )
    return pixel_size * dist_mm / size_mm


guess_focal(images[500], dist_mm=500)

1109.484142727322

In [7]:
focals = []
for dist, im in images.items():
    focals.append(guess_focal(im, dist))
    print(f"{dist: >7} | {focals[-1]: >10.2f}")
avg_focal = np.mean(focals)
print(f"\nAverage: {avg_focal}")

    500 |    1109.48
   1000 |    1107.86
   1500 |    1109.50
   2000 |    1111.20
   2500 |    1111.25
   3000 |    1104.36
   3500 |    1106.17
   4000 |    1097.92
   5000 |    1094.02
   6000 |    1104.57

Average: 1105.632015069326


In [8]:
print(focals)
np.std(focals)

[1109.484142727322, 1107.8561147054036, 1109.4975471496582, 1111.197895473904, 1111.2467447916667, 1104.363203048706, 1106.1698065863716, 1097.9162851969402, 1094.021201133728, 1104.5672098795574]


5.427784639649175

In [9]:
test_cam_mat = np.array(
    [
        [avg_focal, 0, 0],
        [0, avg_focal, 0],
        [0, 0, 1],
    ]
)

In [10]:
images[500]

array([[[ 77,  76,  92],
        [ 75,  74,  90],
        [ 79,  76,  92],
        ...,
        [140, 122, 129],
        [145, 127, 134],
        [160, 142, 149]],

       [[ 79,  78,  94],
        [ 78,  77,  93],
        [ 77,  74,  90],
        ...,
        [141, 123, 130],
        [157, 139, 146],
        [171, 153, 160]],

       [[ 73,  74,  88],
        [ 72,  73,  87],
        [ 72,  71,  87],
        ...,
        [146, 126, 133],
        [163, 143, 150],
        [173, 153, 160]],

       ...,

       [[ 99,  81,  90],
        [103,  85,  94],
        [102,  83,  96],
        ...,
        [ 47,  43,  59],
        [ 47,  44,  58],
        [ 47,  44,  58]],

       [[100,  80,  89],
        [104,  84,  93],
        [104,  83,  95],
        ...,
        [ 45,  41,  57],
        [ 46,  42,  56],
        [ 46,  42,  56]],

       [[100,  80,  89],
        [100,  80,  89],
        [101,  80,  92],
        ...,
        [ 46,  42,  58],
        [ 46,  42,  56],
        [ 46,  42,  56]]

In [11]:
_object_points = np.array(
    [
        [-marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, marker_size_mm / 2, 0],
        [marker_size_mm / 2, -marker_size_mm / 2, 0],
        [-marker_size_mm / 2, -marker_size_mm / 2, 0],
    ]
)


def estimate_pose(cn, cam_internal=test_cam_mat, obj_points=_object_points):
    ret, r_vec, t_vec = cv2.solvePnP(
        obj_points,
        imagePoints=cn,
        cameraMatrix=cam_internal,
        distCoeffs=np.zeros((4,)),
        flags=cv2.SOLVEPNP_IPPE_SQUARE,
    )
    assert ret, "Failed to solve PnP for pose estimation!"
    r_mat = np.empty((3, 3))
    cv2.Rodrigues(r_vec, dst=r_mat)
    return r_mat, t_vec


estimate_pose(corners[0][0])

(array([[ 0.99987173,  0.01598344, -0.00102674],
        [ 0.01598207, -0.99987139, -0.00133023],
        [-0.00104787,  0.00131365, -0.99999859]]),
 array([[2344.26555417],
        [1932.78990169],
        [5054.89266862]]))